In [11]:
!pip3 install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 30.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from pytube import YouTube
import os

In [25]:
yt_string = "https://www.youtube.com/watch?v=CGUFF7aXjTw"
path = "/Users/trishitasingh/Desktop"

yt = YouTube(yt_string)

In [28]:
video = yt.streams.filter(only_audio=True).first()

In [29]:
out_file = video.download(output_path=path)

base, ext = os.path.splitext(out_file)
new_file = base + '.mp3'
os.rename(out_file, new_file)

In [31]:
# result of success
print(yt.title + " has been successfully downloaded.")
audio_file_path = path + "/" + yt.title + ".mp3"

image_file_path = path+"/"+'output_with_circle.avi'

output_path = path+"/"+'output_with_audio_and_circle.mp4'

Breathing (Extended Mix) has been successfully downloaded.


In [32]:
import numpy as np
import librosa
import cv2
from scipy.fft import fft

# Load the audio file
y, sr = librosa.load(audio_file_path, sr=None)

# Parameters for video
frame_rate = 30
video_length = len(y) / sr
num_frames = int(video_length * frame_rate)

# Parameters for frequency extraction
window_size = sr // frame_rate
bass_min = 20
bass_max = 150
high_min = 150

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_size = (640, 480)
out = cv2.VideoWriter(image_file_path, fourcc, frame_rate, video_size)

# Iterate through the audio by window, and create frames
for i in range(num_frames):
    start = i * window_size
    end = (i + 1) * window_size
    
    # Extract frequencies for the current window
    window = np.hanning(window_size)
    segment = y[start:end] * window
    frequencies = np.fft.fftfreq(len(segment), d=1/sr)
    positive_frequencies = frequencies[frequencies >= 0]
    magnitudes = np.abs(fft(segment))[frequencies >= 0]
    bass_magnitudes = magnitudes[(positive_frequencies >= bass_min) & (positive_frequencies <= bass_max)]
    high_magnitudes = magnitudes[positive_frequencies >= high_min]
    
    # Calculate blue intensity based on bass amplitude
    intensity = int(np.mean(bass_magnitudes) * 255 / np.max(magnitudes))
    frame = np.zeros((video_size[1], video_size[0], 3), dtype=np.uint8)
    frame[:, :, 0] = intensity # Set blue channel
    
    # Draw a circle based on high-frequency amplitude
    circle_radius = int(np.mean(high_magnitudes) * video_size[0] / (2 * np.max(magnitudes)))
    cv2.circle(frame, (video_size[0]//2, video_size[1]//2), circle_radius, (255, 255, 255), -1) # Draw white circle
    
    # Write the frame to the video
    out.write(frame)

# Release video writer
out.release()

print("Video with circle created successfully!")

import subprocess


command = ['ffmpeg', '-i', image_file_path, '-i', audio_file_path, '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', output_path]
subprocess.run(command)

[src/libmpg123/parse.c:skip_junk():1276] error: Giving up searching valid MPEG header after 65536 bytes of junk.
/var/folders/9s/38yb0xn536968pgvsk_rgxj40000gp/T/ipykernel_15275/915933488.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file_path, sr=None)
/opt/homebrew/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Video with circle created successfully!


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --dis

CompletedProcess(args=['ffmpeg', '-i', '/Users/trishitasingh/Desktop/output_with_circle.avi', '-i', '/Users/trishitasingh/Desktop/Breathing (Extended Mix).mp3', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', '/Users/trishitasingh/Desktop/output_with_audio_and_circle.mp4'], returncode=0)